In [5]:
import ollama
import mercury as mr

In [17]:
def get_temperature(city: str) -> str:
  """Get the current temperature for a city
  
  Args:
    city: The name of the city

  Returns:
    The current temperature for the city
  """
  temperatures = {
    'New York': '22°C',
    'London': '15°C'
  }
  return temperatures.get(city, 'Unknown')


In [18]:
# list with all user and assistant messages
messages = []

In [19]:
# place to display messages
chat = mr.Chat(placeholder="💬 Start conversation")

In [4]:
# user input
prompt = mr.ChatInput()

In [21]:
if prompt.value:
    # create user message
    usr_msg = mr.Message(markdown=prompt.value, role="user")
    # display user message in the chat
    chat.add(usr_msg)
    # save in messages list
    messages += [{'role': 'user', 'content': prompt.value}]

    # call local LLM
    stream = ollama.chat(
      model='gpt-oss:20b',
      messages=messages,
      tools=[get_temperature],
      stream=True,
    )

    # create assistant message
    ai_msg = mr.Message(role="assistant", emoji="🤖")
    # display assistant message in the chat
    chat.add(ai_msg)

    # stream with response
    thinking, content = "", ""
    tool_calls = []
    for chunk in stream:
        if chunk.message.thinking:
            if thinking == "":
                ai_msg.append_markdown("Thinking: ")    
            thinking += chunk.message.thinking
            ai_msg.append_markdown(chunk.message.thinking)
        elif chunk.message.content:
            if content == "":
                ai_msg.append_markdown("\n\nAnswer: ")    
            content += chunk.message.content
            ai_msg.append_markdown(chunk.message.content)
        elif chunk.message.tool_calls:
            tool_calls.extend(chunk.message.tool_calls)
            
    # save assistant response
    messages += [{'role': 'assistant', 'thinking': thinking, 'content': content, 'tool_calls': tool_calls}]

    for tool in tool_calls:
        if tool.function.name == "get_temperature":
            tool_msg = mr.Message(role="tool", emoji="⛅")
            # display tool message in the chat
            chat.add(tool_msg)
            result = get_temperature(**tool.function.arguments)
            tool_msg.append_markdown("Temperature is " + result)
            messages += [{
                "role": "tool",
                "tool_name": "get_temperature",
                "content": result
            }]    
            
        